In [1]:
# Libraries
import numpy as np
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import re
import os
from urllib.request import urlopen
import time
from concurrent.futures import ThreadPoolExecutor

import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
import json
import heapq

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_geocoder")
import plotly.express as px
import plotly.graph_objects as go

# 1. Data collection

## 1.1. Get the list of master's degree courses
We created a file named 'urls.txt' that contains all the urls associated with the url of each master page.
for reaching such purpose, we iterate over all 400 pages and took the link for every 15 urls of each page.
we stored all urls in 'urls.txt' file.

In [2]:
file_path = "/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-6/ADM-HW3/urls.txt"

if not os.path.exists(file_path):
    with open(file_path, "w") as f:
        for i in range(1, 401):
            url = f"https://www.findamasters.com/masters-degrees/msc-degrees/?PG={i}"
            result = requests.get(url)
            soup = BeautifulSoup(result.text, 'html.parser')

            for link in soup.find_all(class_=re.compile('courseLink')):
                c = link.get("href")
                f.write("https://www.findamasters.com/" + c)
                f.write("\n")
    print('The "urls.txt" file is generated!')
else:
    print('The "urls.txt" file already exists.')

The "urls.txt" file already exists.


### 1.2. Crawl master's degree pages

We wrote a function named 'download_url'.
Since the FindMaster website blocks us for 70 seconds for every (20 to 22) requests we send, we use 'time.sleep(70)' to wait and then resend the http get request. 
we also omit to download the http files that their directory are already existed.

for sending http get requests asynchronously, we can use async and await methods and take the advantage of using "aiohttp" library. the other way is to use ThreadPoolExecutor function executer. 
It means that we store the executer command in a variable named 'future_to_url' that we are able to call in the future.
The ThreadPoolExecutor is a built-in Python module that provides managing a pool of worker threads. It allows us to submit tasks to the pool, which are then executed by one of the worker threads in the pool.

In [4]:
from concurrent.futures import ThreadPoolExecutor
    
# Function to download and save HTML for a given URL
def download_url(url, folder_path, page_number):
    # Create a folder for each page if it doesn't exist
    page_folder = os.path.join(folder_path, f"page_{page_number}")
    if os.path.exists(page_folder):
        # uncomment the below code to see which pages are skiped, cause they have already been downloaded.
        # print(f"Skipping Page: {page_number} - Folder already exists.")
        return

    try:
        response = requests.get(url) # Send a GET request to the URL
        response.raise_for_status()  # Raise an exception for bad responses 

        # Create a folder for each page if it doesn't exist
        os.makedirs(page_folder, exist_ok=True)

        # Save the HTML content to a file
        file_path = os.path.join(page_folder, f"html_{page_number}.html")
        if not os.path.exists(file_path):
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"Downloaded page {page_number}: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download page {page_number}: {url}")
        print(f"Error: {e}")
        print("Retrying in 70 seconds...")
        time.sleep(70)  # Wait for 10 seconds before retrying
        download_url(url, folder_path, page_number)  # Retry the download

# Read all URLs one by one
with open('urls.txt', 'r') as urls_file:
    urls = urls_file.read().splitlines()

output_folder = 'HTML_folders' # Store all HTML files into this directory.

# We can use ThreadPoolExecutor for sending http requests asynchronously. 
# However, Since the FindMaster website blocks us for 70 seconds for every (20 to 22) requests we send, 
# the max_workers in below code assigned to number 1. So it sends requests synchronously.
with ThreadPoolExecutor(max_workers=1) as executor:
    # Enumerate through each URL and submit download tasks to the executor
    future_to_url = {executor.submit(download_url, url, output_folder, page_number): url for page_number, url in enumerate(urls, start=1)}

print("All HTML files are stored in the HTML_folders directory.")

All HTML files are stored in the HTML_folders directory.


### 1.3 Parse downloaded pages
Here we create a '.tsv' file including the following columns for each of the HTML files.

1. Course Name (to save as ```courseName```): string;
2. University (to save as ```universityName```): string;
3. Faculty (to save as ```facultyName```): string
4. Full or Part Time (to save as ```isItFullTime```): string;
5. Short Description (to save as ```description```): string;
6. Start Date (to save as ```startDate```): string;
7. Fees (to save as ```fees```): string;
8. Modality (to save as ```modality```):string;
9. Duration (to save as ```duration```):string;
10. City (to save as ```city```): string;
11. Country (to save as ```country```): string;
12. Presence or online modality (to save as ```administration```): string;
13. Link to the page (to save as ```url```): string.

Then, we merge all those files together to generate our final dataset.

In [6]:
current_path = os.getcwd()
# '/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-2/ADM-HW3/HTML_folders'

for i in range(1,6001):
    # os.chdir(r'C:\Users\susan\Documents\DS\ADM\HW3\ADM-HW3\HTML_folders\page_'+str(i)) #change directories
    os.chdir(r'/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-3/ADM-HW3/HTML_folders/page_'+str(i)) #change directories
    
    for filename in os.listdir(os.getcwd()): # get all the files in a folder
        if filename.endswith(".tsv"): continue # tsv file is already generated.
        elif filename.endswith(".html"): # if file extension is .html
            with open(os.path.join(os.getcwd(), filename), 'r',encoding='utf-8') as f: # open each file into a folder
                soup = BeautifulSoup(f,'html.parser') # get the html file by each file 
                out=[] # initialize a list where we append all the informations parsed from each html file

                # 1  Course Name
                courseName = soup.find_all(class_=re.compile("course-header__course-title"))
                out.append(courseName[0].text.strip() if courseName else "") #text.strip to eliminate strange simbols for the space
                # 2  University
                universityName = soup.find_all(class_=re.compile("course-header__institution"))
                out.append(universityName[0].text if universityName else "")
                # 3  Faculty
                facultyName = soup.find_all(class_=re.compile("course-header__department"))
                out.append(facultyName[0].text if facultyName else "")
                # 4  Full or Part Time
                isItFullTime = soup.find_all(class_=re.compile("concealLink"))
                out.append(isItFullTime[0].text if isItFullTime else "")
                # 5  Short Description
                description = soup.find_all(class_=re.compile("course-sections__content"))
                out.append(description[0].text.replace('\n', '') if description else "")
                # 6  Start Date
                startDate = soup.find_all(class_=re.compile("key-info__start-date"))
                out.append(startDate[0].text if startDate else "")
                # 7  Fees 
                fees_elements = soup.find_all(class_=re.compile("course-sections__fees")) # taking the fee
                fees_text = fees_elements[0].text.replace('\n', '') if fees_elements else "" 
                cleaned_fees = re.sub(r'Fees', '', fees_text)  # To not "Fees" at the beginning 
                out.append(cleaned_fees.strip() if cleaned_fees else "")
                # 8  Modality
                modality = soup.find_all(class_=re.compile("key-info__qualification"))
                out.append(modality[0].text if modality else "")
                # 9  Duration
                duration = soup.find_all(class_=re.compile("key-info__duration"))
                out.append(duration[0].text if duration else "")
                # 10  City
                city = soup.find_all(class_=re.compile("course-data__city"))
                out.append(city[0].text if city else "")
                # 11  Country
                country = soup.find_all(class_=re.compile("course-data__country"))
                out.append(country[0].text if country else "")
                # 12  Presence or online modality
                # We have seen that some courses has both online or oncampus modality, one of them is "Master of Business Administration"
                on_campus_elements = soup.find_all(class_=re.compile("course-data__on-campus"))
                online_elements = soup.find_all(class_=re.compile("course-data__online"))
                if on_campus_elements and online_elements:
                    out.append("both")
                else:
                    out.append(on_campus_elements[0].text if on_campus_elements else online_elements[0].text if online_elements else "Nan")
                # 13  Link to the page
                out.append(soup.find('link', {'rel': 'canonical'}).get('href') if soup.find('link', {'rel': 'canonical'}) else "Nan")
                f.close()
                
                # Creating file .tsv
                l = ['courseName','universityName','facultyName','isItFullTime','description','startDate','fees','modality','duration',
                    'city','country','administration','url']
                with open(filename+'.tsv','w',encoding='utf-8') as tsv:
                    tsv_output = csv.writer(tsv, delimiter='\t')
                    tsv_output.writerow(l)
                    tsv_output.writerow(out)
    os.chdir('..')  

print("All HTML files have been read and all .tsv files have been generated.")

All HTML files have been read and all .tsv files have been generated.


In [8]:
data=[]
# to merge all the .tsv files
for i in range(1,6001):
    # os.chdir(r'./HTML_folders/page_'+str(i)) #change directories
    os.chdir(r'/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-3/ADM-HW3/HTML_folders/page_'+str(i)) #change directories
    for filename in os.listdir(os.getcwd()):
        if filename.endswith(".tsv"):
            a = pd.read_csv(filename,sep='\t')
            data.append(a)
    os.chdir('..')
data=pd.concat(data,ignore_index=True)   
data.to_csv('../dataset.tsv',sep='\t',index=False) # Saving the big one
print("dataset.tsv file has been generated as the main dataset.")

dataset.tsv file has been generated as the main dataset.


## 2. Search Engine

For the search engine, we loaded the dataset obtained in step 1 and then started preprocessing. \
We decided to clean only the description column because we did not think it would be useful to clean for example also the mastrer degree name because with the removal of the stop words and stemming it would no longer be possible to figure out the real name of the degree.

In [2]:
# Load the dataset we are going to work with
data = pd.read_table(r"dataset.tsv")
# Drop NAs
data = data.dropna()
data

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Full time,Register your interest in graduate study at UC...,September,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5996,Materials Chemistry - MSc,University of Bradford,Faculty of Life Sciences,Full time,We provide a unique Master’s education in Mate...,September,Please see the university website for further ...,MSc,1 year full time,Bradford,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5997,Materials Chemistry MSc,University of Edinburgh,School of Chemistry,Full time,Programme descriptionMaterials Chemistry has e...,September,Tuition fees vary between degree programmes. F...,MSc,1 year full-time,Edinburgh,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5998,Materials Engineering,University of Padua,School of Engineering,Full time,The Master's degree Materials Engineering is a...,October,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,On Campus,https://www.findamasters.com/masters-degrees/c...


## 2.0 Preprocessing

### 2.0.0) Preprocessing the text


In [3]:
# This function takes a text, removes special cases, punctuations and stop-words then
# it applies stemming and finally returns the preprocessed words separated by commas.

def preprocess_description(description_text):

    # Handle the cases of float type (there are 24)
    if type(description_text) != str:
        return ""

    # Remove all special chars and punctuations
    description_text = re.sub("[^a-z A-Z ]+","", description_text)

    # Convert everything in lowercase
    description_text = description_text.lower()

    # Remove stopwords using nltk package
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(description_text)
    words = [word for word in words if word not in stop_words]

    # Apply stemming using ntlk package
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Separate words with commas
    words = ','.join(words)

    return words

In [4]:
# Create and fill new column where the preprocessed descriptions wil be stored:
data["clean_description"] = data["description"].apply(preprocess_description)

In [5]:
# Show different fields
data[["description", "clean_description"]].head(10)

,description,clean_description
0,3D visualisation and animation play a role in ...,"visualis,anim,play,role,mani,area,popular,medi..."
1,Businesses and governments rely on sound finan...,"busi,govern,reli,sound,financi,knowledg,underp..."
2,"Our Accounting, Accountability & Financial Man...","account,account,financi,manag,msc,cours,provid..."
3,Embark on a professional accounting career wit...,"embark,profession,account,career,academ,ground..."
4,Join us for an online session for prospective ...,"join,us,onlin,session,prospect,student,find,ms..."
5,The Advanced Chemical Engineering MSc at Leeds...,"advanc,chemic,engin,msc,leed,build,core,founda..."
6,Programme overviewThe Advanced Master in Finan...,"programm,overviewth,advanc,master,financi,mark..."
7,Programme overviewThe Advanced Master in Innov...,"programm,overviewth,advanc,master,innov,strate..."
8,Progress your career as a physiotherapist with...,"progress,career,physiotherapist,within,nh,priv..."
9,Goal of the pro­grammeWould you like to be inv...,"goal,programmewould,like,involv,find,solut,fut..."


### 2.0.1) Preprocessing the fees column

When it comes to preprocessing the tax column, we obtained the exchange rates thanks to a function asked of ChatGPT.  \
 In the _preprocess_fees_ function we searched with regexes all the various costs of the universities and then turned them all into dollar USD prices by dividing the cost we found by the exchange rates.

In [6]:
# This function was provided by ChatGPT.
# It downloads the latest exchange rates (wrt United States Dollars) from openexchangerates.org.
# It returns a dictionary of the form {exchange_code : exchange_rate}
def get_latest_exchange_rates(app_id):

    base_url = "https://openexchangerates.org/api/latest.json"
    params = {"app_id": app_id}
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json().get("rates")
    else:
        print(f"Failed to fetch exchange rates. Status code: {response.status_code}")
        return None

In [7]:
# Load latest exchange rates
my_app_id = "1457fcd3d536441baad3ce7918b5025b"
exchange_rates_dict = get_latest_exchange_rates(my_app_id)

# Show exchange_rates_dict
count = 0
print("{")
for key, value in exchange_rates_dict.items():
    if count < 5:
        print(f'{key}: {value}')
        count += 1
    else:
        break
print("}")

{
AED: 3.672625
AFN: 69.193188
ALL: 93.466047
AMD: 401.077237
ANG: 1.796265
}


In [8]:
# Dictionary provided by ChatGPT, I changed some symbols that were not present.
# It will be used to convert a currency symbol into currency code.

currency_symbol_to_code = {
    '$': 'USD',  # United States Dollar
    '€': 'EUR',  # Euro
    "EURO": "EUR", # Euro
    '¥': 'JPY',  # Japanese Yen
    '£': 'GBP',  # British Pound Sterling
    'A$': 'AUD',  # Australian Dollar
    'C$': 'CAD',  # Canadian Dollar
    'CHF': 'CHF',  # Swiss Franc
    'KR': 'SEK',  # Swedish Krona
    'NZ$': 'NZD',  # New Zealand Dollar
    # Add more symbols and codes as needed
}

# This function handles the preprocessing of the "fees" field
def preprocess_fees(fees_text):

    # Handles the case of "nan"
    if type(fees_text) != str:
        return None

    # Preallocate all the fees found in text
    total_fees = []

    # Symbols we are looking for
    symbols = "GBP|USD|ISK|£|\$|₹|¥|₪|₽|₩|₦|₴|﷼|€|Euro"

    # Match the (symbol, number) case
    left_symbol_matches = re.findall(fr'(?:{symbols})+\s*[0-9]+[,.]?[0-9]*', fees_text, flags=re.IGNORECASE)

    # Match the (number,  symbol) case
    right_symbol_matches = re.findall(fr'[0-9]+[,.]?[0-9]*\s*(?:{symbols})+', fees_text, flags=re.IGNORECASE)

    # Merge them
    matches = left_symbol_matches + right_symbol_matches

    # If we got no matches returns None
    if len(matches) == 0 :
        return None

    for match in matches:
        # Remove "," or "." and change to the right type
        number = re.findall('([0-9]+)[.,]*([0-9]*)', match)[0]
        number = float(number[0] + number[1])

        # Isolate symbol and upper case it (to match exchange_rates_dict codes)
        symbol = re.findall(fr'(?i)({symbols})', match)[0].upper()

        # Transform symbol into code (if not already a code)
        if symbol in currency_symbol_to_code.keys():
            symbol = currency_symbol_to_code[symbol]

        # Change into USD using the exchange_rates dictionary and append to fees
        total_fees.append(number / exchange_rates_dict[symbol])

    # Take the max fee and return it
    max_fee = round(max(total_fees))
    return max_fee


<>:29: SyntaxWarning: invalid escape sequence '\$'
<>:29: SyntaxWarning: invalid escape sequence '\$'
/var/folders/d6/sz5pbmks2gs12gkp2yzr1gl00000gn/T/ipykernel_52759/1548670465.py:29: SyntaxWarning: invalid escape sequence '\$'
  symbols = "GBP|USD|ISK|£|\$|₹|¥|₪|₽|₩|₦|₴|﷼|€|Euro"


In [9]:
# Save preprocessed fees into a new column, and show them
data["fees_USD"] = data["fees"].apply(preprocess_fees)
data[["fees", "fees_USD"]].head(10)

,fees,fees_USD
0,Please see the university website for further ...,NaN
1,"UK: £18,000 (Total) International: £34,750 (To...",43788.0
2,Please see the university website for further ...,NaN
3,Please see the university website for further ...,NaN
4,Please see the university website for further ...,NaN
5,"UK: £13,750 (Total) International: £31,000 (To...",39063.0
6,18.000 €,19703.0
7,18.000 €,19703.0
8,Please see the university website for further ...,NaN
9,Tuition fee per year (non-EU/EEA students): 15...,16419.0


## 2.1. Conjunctive query

### 2.1.1) Create your index!

For the conjunctive query we followed the commands received and then first constructed a _terms_id_dict_ dictionary where each term is associated with a unique id, we then created the inverted index and showed its structure for the first 5 rows. \
With the _naive_search_engine_ function, we obtained the results by selecting the document ids that contain all the words in the query.

In [10]:
# Store all terms contained in the "clean_description" as a set
terms_set = set(','.join(data["clean_description"]).split(","))

# Create a dict that associate each term to a unique id
terms_id_dict = {key: value for value, key in enumerate(terms_set)}

In [11]:
# Here we create the inverted index dictionary.

# Preallocate a dictionary with the form: {term_id : []}
inverted_index = {i : [] for i in range(len(terms_id_dict))}

# Iterating over all terms and texts in the "clean_description" field
for i,text in enumerate(data["clean_description"]):

    text_set = set(text.split(","))

    for term in text_set:
        # Get term id
        term_id = terms_id_dict[term]

        # Add document id "i" to the term_id list
        inverted_index[term_id].append(i)

In [12]:
# Show inverted_index_dict structure for top 5
print("{")
count = 0
for key, value in inverted_index.items():
    if count < 5:
        print(f'{key}: {value}')
        count += 1
    else:
        break
print("}")

{
0: [95, 151, 394, 400, 417, 476, 659, 660, 683, 882, 904, 1036, 1064, 1065, 1143, 1154, 1159, 1183, 1189, 1423, 1496, 1779, 1784, 1853, 1903, 2117, 2161, 2302, 2456, 2485, 2499, 2558, 2953, 3131, 3138, 3139, 3140, 3141, 3142, 3143, 3190, 3371, 3419, 3437, 3710, 3736, 3827, 3964, 4047, 4086, 4208, 4320, 4330, 4636, 4723, 4733, 4981, 5082, 5388, 5579, 5640, 5794, 5814]
1: [1859]
2: [1876]
3: [700, 3710, 4733, 5388]
4: [5320]
}


### 2.1.2) Execute the query

In [13]:
# This function takes a query as a input and returns the most affine docs:

def naive_search_engine(query):
    # Apply same preprocessing done for descriptions and split wrt ","
    query = preprocess_description(query).split(",")

    # For each term in query get all the docs ids that contain it as a set
    query_docs = [set(inverted_index[terms_id_dict[term]]) for term in query]

    # Select the docs ids that contain all the query term, and sort them
    query_docs = set.intersection(*query_docs)
    query_docs = list(sorted(query_docs))

    # Return selected columns of those docs
    result  = data.iloc[query_docs, [0,1,4,12]]

    return result

In [14]:
naive_search_engine("advanced knowledge")

,courseName,universityName,description,url
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
4,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...
12,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
48,Civil Engineering MSc,University of Greenwich,Meet the future demands of the construction in...,https://www.findamasters.com/masters-degrees/c...
86,Economics - MSc,University of Leeds,Our MSc Economics allows you to apply economic...,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...
5909,Master of Science/Postgraduate Diploma in Envi...,The Hong Kong University of Science and Techno...,The program is meant to meet the needs of prac...,https://www.findamasters.com/masters-degrees/c...
5937,Master Sociology – Social and Economic Psychology,University of Cologne,This programme provides you with:a solid found...,https://www.findamasters.com/masters-degrees/c...
5957,Masters in Economics,University of Lisbon,OBJECTIVESThe MSc in Economics aims to provide...,https://www.findamasters.com/masters-degrees/c...
5963,Master's in Global and European Politics,European School of Political and Social Scienc...,Europe and the EU in a changing worldOur inter...,https://www.findamasters.com/masters-degrees/c...


## 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index


In [15]:
# Here we create the second inverted index dictionary.

# Preallocate a dictionary with the form: {term_id : [(doc_id, tfidf_score)]}
new_inverted_index = {i : [] for i in range(len(terms_id_dict))}

# Iterating over all terms and texts in the "clean_description" field
for i, text in enumerate(data["clean_description"]):

    text_list = text.split(",")

    # "set" here has the purpose of selecting unique terms only.
    for term in set(text_list):

        # Get term id
        term_id = terms_id_dict[term]

        # Get idf score: log( total number of documents / number of documents term is in )
        term_idf = np.log(len(data["clean_description"]) / len(inverted_index[term_id]))

        # Get tf score: number of times term appears in text / total number of terms in text
        term_tf =  text_list.count(term) / len(text_list)

        # Compute tfidf score
        term_tfidf = term_tf * term_idf

        # Update new_inverted_index list with a tuple
        new_inverted_index[term_id].append((i , term_tfidf))

In the execution of queries  we will use the inverted index dictionary defined below, which transforms the list of tuples contained in new_inverted_index into a dictionary.
With this structure **tfidf(term_id, doc_id) = new_inverted_index_as_dict[term_id][doc_id]**

In [16]:
# Convert new_inverted_index values to dictionaries
new_inverted_index_as_dict = {key : dict(new_inverted_index[key]) for key in new_inverted_index.keys()}

## 2.2.2) Execute the query

We define our second search engine below

In [17]:
# This function computes from scratch cosine similarity
def cosine_similarity(vec1, vec2):

    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # Handle division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0

    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def  top_k_search_engine(query, k):

    # Apply same preprocessing done for descriptions and split wrt ","
    query = preprocess_description(query).split(",")

    # Get query terms ids
    query_terms_ids = [terms_id_dict[term] for term in set(query)]

    # Calculate for each term in query its tfidf score
    query_terms_idf = np.array([np.log(len(data["clean_description"]) / len(inverted_index[term_id])) for term_id in query_terms_ids])
    query_terms_tf =  np.array([query.count(term) / len(query) for term in set(query)])
    query_terms_tfidf = query_terms_idf * query_terms_tf

    # Create the tfidf representation of the query
    query_tfidf = np.zeros(len(terms_set))
    query_tfidf[query_terms_ids] = query_terms_tfidf

    # Get the indexes of docs that contain all terms in query using the first inverted index dict
    docs_ids = [set(inverted_index[term_id]) for term_id in query_terms_ids]
    appropriate_docs_ids = list(set.intersection(*docs_ids))

    # docs_tfidf will contain for each doc its tfidf vectorial representation.
    docs_tfidf = {i : [] for i in appropriate_docs_ids}

    # Iterate over docs containing all the words in query
    for doc_id in appropriate_docs_ids:

        # Initialize tdidf vectorial representation of doc
        doc_tfidf = np.zeros(len(terms_set))

        # Get doc as a list of preprocessed words
        doc = data["clean_description"].iloc[doc_id].split(",")

        # For each term
        for term in set(doc):
            # Get term_id
            term_id = terms_id_dict[term]

            # Store tfidf(term_id, doc_id) in doc vectorial representation
            doc_tfidf[term_id] = new_inverted_index_as_dict[term_id][doc_id]

        # Store doc vectorial representation
        docs_tfidf[doc_id] = doc_tfidf


    # Compute cosine similarities between query_tfidf and each doc_tfidf
    cos_sims = [cosine_similarity(query_tfidf, docs_tfidf[key]) for key in appropriate_docs_ids]


    # Select all appropriate_docs_ids from data and specified columns
    result  = data.iloc[appropriate_docs_ids, [0,1,4,12]]

    # Add the cosine similarity score and sort the dataframe
    result["cos_sim"] = cos_sims
    result = result.sort_values(by='cos_sim', ascending=False)

    # Get, if possible, just the top k
    if k < result.shape[0]:
        result = result[:k]

    return result

In [26]:
top_k_search_engine("advanced knowledge", k = 10)

,courseName,universityName,description,url,cos_sim
756,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,https://www.findamasters.com/masters-degrees/c...,0.361235
701,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for lea...,https://www.findamasters.com/masters-degrees/c...,0.333983
931,Advancing Practice - MSc,University of Northampton,Our MSc Advancing Practice awards support the ...,https://www.findamasters.com/masters-degrees/c...,0.325750
654,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,https://www.findamasters.com/masters-degrees/c...,0.283256
830,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,This course offers a broad range of advanced s...,https://www.findamasters.com/masters-degrees/c...,0.279986
897,Advanced Professional Practice (MSc),University of Gloucestershire,Our lecturers are research active experts who ...,https://www.findamasters.com/masters-degrees/c...,0.270317
653,Advanced Clinical Practice - MSc,University of Northampton,Our MSc Advanced Clinical Practice course aims...,https://www.findamasters.com/masters-degrees/c...,0.270123
786,Advanced Healthcare Practice - MSc,Cardiff University,Why study this courseOur MSc Advanced Healthca...,https://www.findamasters.com/masters-degrees/c...,0.267574
712,Advanced Clinical Practitioner - MSc,University of Sunderland,The MSc Advanced Clinical Practitioner is a hi...,https://www.findamasters.com/masters-degrees/c...,0.251893
892,Advanced Practice in Healthcare MSc,University of Liverpool,Explore specialist areas of practice in-depth ...,https://www.findamasters.com/masters-degrees/c...,0.250338


## 3. Define a new score!
Now it's your turn: build a new metric to rank MSc degrees.

### Answer

To develop our customized search engine, we opted to gather supplementary information for each university.
Recognizing that the selection of the "ideal" university is influenced by the city in which one resides, we chose to extract scores reflecting the quality of life in each city.

The following function used the [Teleport](https://teleport.org/) API to retrieve, for each city in our Data, three scores:
- Education score;
- Safety score;
- Cost of living score.

All of them are normalized in $[0,10]$.
When a city was not integrated into the teleport API, our solution was to obtain the scores associated with the capital of the country in which the city is located. In order to do so we will use the country field of our data in union with the "country_capital_dict", which maps each country to its capital.

In [28]:
# Load country_capital_dict
with open("country_capital_dict.json", 'r') as json_file:
    country_capital_dict = json.load(json_file)

# This function returns life quality scores for each city
def get_scores_out_of_ten(city):
    # Handle different data types
    city = str(city)

    try:
        # Try to get URL of the initial city
        base_url = f"https://api.teleport.org/api/urban_areas/slug:{city.lower()}/scores/"
        response = requests.get(base_url)

        # If city not found, retrieve URL of capital city
        if response.status_code == 404:

            # Retrieve the country of the city
            country = data[data["city"] == city]["country"].iloc[0]

            # Retrieve capital city from  the country
            capital_city = country_capital_dict[country]

            # Try to get URL of the capital city
            base_url = f"https://api.teleport.org/api/urban_areas/slug:{capital_city.lower()}/scores/"
            response = requests.get(base_url)

            # If capital city not found, return error
            if response.status_code == 404:
                return ("City not found", "City not found", "City not found")

        # Retrieve scores from URL
        datas = response.json()

        # Retrieve all categories scores
        categories = datas['categories']
        scores = {category["name"] : float(category["score_out_of_10"]) for  category in categories}

        # Return Cost of Living, Education and Safety
        return (scores["Cost of Living"],
                scores["Education"],
                scores["Safety"])


    except requests.exceptions.RequestException as e:
        return {'error': f'Request to Teleport API failed: {e}'}

In [ ]:
# Create three new columns for our new scores (around 10m to run)
data[['Cost_of_Living', 'Education', 'Safety']] = data.city.apply(lambda x: pd.Series(get_scores_out_of_ten(x)))

In [30]:
# Show the new columns
data[['city','Cost_of_Living', 'Education', 'Safety']].head(10)

,city,Cost_of_Living,Education,Safety
0,Glasgow,5.623,5.3065,7.4960
1,Leeds,5.363,4.9825,7.7310
2,London,3.940,9.0270,7.2435
3,Reading,3.940,9.0270,7.2435
4,London,3.940,9.0270,7.2435
5,Leeds,5.363,4.9825,7.7310
6,Brussels,4.477,6.6530,6.7030
7,Brussels,4.477,6.6530,6.7030
8,Glasgow,5.623,5.3065,7.4960
9,Helsinki,4.121,5.4545,8.6740


Those scores will affect the importance our custom search engine gives to each university.
As an initial and naive approach we decided to keep thinks linear and just compute the new score as shown below:

\begin{equation}
NewScore = \frac{CosSimilarity \cdot Education \cdot Safety }{LivingCost}
\end{equation}


The following function follows the one already shown previously but orders the result with respect to the new score we just defined.

In [207]:
data = pd.read_table("new_dataset.tsv")

In [208]:
def  custom_search_engine(query, k):

    # Apply same preprocessing done for descriptions and split wrt ","
    query = preprocess_description(query).split(",")

    # Get query terms ids
    query_terms_ids = [terms_id_dict[term] for term in set(query)]

    # Calculate for each term in query its tfidf score
    query_terms_idf = np.array([np.log(len(data["clean_description"]) / len(inverted_index[term_id])) for term_id in query_terms_ids])
    query_terms_tf =  np.array([query.count(term) / len(query) for term in set(query)])
    query_terms_tfidf = query_terms_idf * query_terms_tf

    # Create the tfidf representation of the query
    query_tfidf = np.zeros(len(terms_set))
    query_tfidf[query_terms_ids] = query_terms_tfidf

    # Get the indexes of docs that contain all terms in query using the first inverted index dict
    docs_ids = [set(inverted_index[term_id]) for term_id in query_terms_ids]
    appropriate_docs_ids = list(set.intersection(*docs_ids))

    # docs_tfidf will contain for each doc its tfidf vectorial representation.
    docs_tfidf = {i : [] for i in appropriate_docs_ids}

    # Iterate over docs containing all the words in query
    for doc_id in appropriate_docs_ids:

        # Initialize tdidf vectorial representation of doc
        doc_tfidf = np.zeros(len(terms_set))

        # Get doc as a list of preprocessed words
        doc = data["clean_description"].iloc[doc_id].split(",")

        # For each term
        for term in set(doc):
            # Get term_id
            term_id = terms_id_dict[term]

            # Store tfidf(term_id, doc_id) in doc vectorial representation
            doc_tfidf[term_id] = new_inverted_index_as_dict[term_id][doc_id]

        # Store doc vectorial representation
        docs_tfidf[doc_id] = doc_tfidf


    # Compute cosine similarities between query_tfidf and each doc_tfidf
    cos_sims = [cosine_similarity(query_tfidf, docs_tfidf[key]) for key in appropriate_docs_ids]


    # Select all appropriate_docs_ids from data and specified columns
    result  = data.iloc[appropriate_docs_ids, [0,1,4,12, -3, -2, -1]]

    # Add the cosine similarity score
    result["Cos_sim"] = cos_sims

    # Compute our custom score
    our_score = result["Cos_sim"] * result["Education"] * result["Safety"] / result["Cost_of_Living"]

    # Add the cosine similarity score 
    result["Score"] = our_score

    # Convert the DataFrame to a dictionary
    diz = result.to_dict()

    # Mantaining the top k-university into an heap
    l_heap = []
    # Iterate over the keys (indices) and values (scores) 
    for i,j in zip(diz["courseName"].keys(),diz["Score"].values()):
        l_1=[]
        l_1.append(j) # Append the score
        l_1.append(i) # Append the index (university ID)
        heapq.heappush(l_heap,l_1) # Push the pair (score, index) onto the heap

    doc=[] # List to store the indices of the top K univerisities
    score=[] # List to store the corresponding scores of the top K universities

    # Retrieve the top K elements from the heap
    for i in heapq.nlargest(k,l_heap):
        doc.append(i[1])
        score.append(i[0])

    # Select rows from the result datafram corresponding to the top K universities
    df_heap = result.loc[doc]

    # Return the DataFrame containing information about the top K universities
    return df_heap

In [209]:
custom_search_engine("advanced knowledge", k = 10)

,courseName,universityName,description,url,Cost_of_Living,Education,Safety,Cos_sim,Score
5820,Masters in Hospitality Management,Ecole hotelière de Lausanne,"With our Master in Hospitality Management, you...",https://www.findamasters.com/masters-degrees/c...,1.00,6.622,8.0325,0.113095,6.015666
738,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.361235,5.994941
685,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for lea...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.333983,5.542688
906,Advancing Practice - MSc,University of Northampton,Our MSc Advancing Practice awards support the ...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.325750,5.406058
639,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.283256,4.700829
875,Advanced Professional Practice (MSc),University of Gloucestershire,Our lecturers are research active experts who ...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.270317,4.486097
638,Advanced Clinical Practice - MSc,University of Northampton,Our MSc Advanced Clinical Practice course aims...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.270123,4.482883
696,Advanced Clinical Practitioner - MSc,University of Sunderland,The MSc Advanced Clinical Practitioner is a hi...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.251893,4.180335
607,Advanced Biomedical Engineering - MSc,University of Bradford,Biomedical engineering is a fast evolving inte...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.243003,4.032800
655,Advanced Clinical Practice (AHP) - MSc/PGDip/P...,Bangor University,The programme has been developed to enhance pr...,https://www.findamasters.com/masters-degrees/c...,3.94,9.027,7.2435,0.230842,3.830989


## 4. Visualizing the most relevant MSc degrees

Given the results of step 3 we selected the city and country of the results, thanks to the _geopy_ library we derive the longitude and latitude of the cities of the master's degree programs. \
Then with the _plotly_ library we constructed an interactive map in which the dots are the different courses of study given by the query result. In the map we showed the course name, faculty, university name, fees and the three new indices derived in step 3 which are safety, education and cost of living.  \
The result points are also colored according to fees with a scale from yellow (high), fuchsia (medium) to blue (low).

In [210]:
# Select only the data from the search engine above
todisplay = custom_search_engine("advanced knowledge", k = 40)

In [211]:
# First we create a variable where we store the cities and their corresponding country
our_places = set(zip(data["city"],data["country"]))

# Retrieve the coordinates
coordinates = []

for city, country in our_places:
    location = geolocator.geocode(f"{city}, {country}")
    if location is not None:
        coordinates.append((city, country, location.latitude, location.longitude))

# Initialize the columns to None
data["latitude"] = None
data["longitude"] = None

# Let's put them into our original dataframe
for city, country, latitude, longitude in coordinates:
    mask = (data["city"] == city) & (data["country"] == country)
    data.loc[mask, "latitude"] = latitude
    data.loc[mask, "longitude"] = longitude

# Selecting the right indexes
places = data.iloc[list(todisplay.index),:]

In [222]:
# This is a key from mapbox.com necessary to use the style "open-street-map"
px.set_mapbox_access_token("pk.eyJ1Ijoic3VzYW5uYWJyYXZpIiwiYSI6ImNscGQ3bXR5eTEwamoya3FyaTBjbWt2N2wifQ.h95oGTEioKMBjxmsSAF3yw")

# Plotting the results
fig = px.scatter_mapbox(places, lat="latitude", lon="longitude", hover_name="courseName", hover_data=["fees_USD","Safety","Education","Cost_of_Living","courseName","universityName","facultyName"], color = "fees_USD",
                         zoom=5, height=600, mapbox_style="open-street-map")
fig.show()

We also add a screen of our map in case GitHub does not supports interactive maps
![Immagine](./screen-shots/map.png)

## 5. BONUS: More complex search engine

The following pice of code, read the dataset, and then accepts queries from user in 2 ways.
1. User can search dataset based on 'course_name, university_name, university_city, fee_min, fee_max, countries, started_only, online_only'. User can specify none, all or some of them to retrive the desired rows.
2. User can Write a sentence as query that searches through course_name, university_name, university_city, and based on the most similarity, the data will be returned. For the other options, user should explicitly specify them as argument to the search_courses() function.

In second approach, in order to find the similarity,
- We first, tokenize the text values in course_name, university_name, university_city columns.
- We put tokens in 'inverted_indexes' to find where these tokens occur in the dataset or documents.
- Then we pass query, inverted_indexes, data as arguments to retrieve_similar_documents function. Here we tokenize the query, we finally need to returns a list of document IDs sorted by similarity scores. So, we Use the inverted indexes to find relevant documents by comparing tokens in the query with those in the indexed documents.
- We then, calculate cosine similarity scores between the query and each relevant document.
- In cosine_similarity function, we create two frequency vectors including 'query_vector' and 'doc_vector' to represent the occurrences of tokens in the query and document. It calculates the dot product between these vectors and the cosine similarity
- Finnaly, we sort the documents by their similarity scores.

In [54]:
import pandas as pd
from datetime import datetime

# Load the dataset we are going to work with
data = pd.read_table(r"dataset.tsv")  # Read the dataset from a tab-separated file

from collections import defaultdict  # Import defaultdict from the collections module
import math  # Import the math module for mathematical operations

def tokenize(text):
    if isinstance(text, str):
        return text.lower().split()  # Convert text to lowercase and split into tokens
    return []  # Return an empty list for non-string values

# Function to calculate cosine similarity
def cosine_similarity(query_tokens, doc_tokens):
    query_vector = defaultdict(int)  # Create a dictionary to store query token frequencies
    doc_vector = defaultdict(int)  # Create a dictionary to store document token frequencies

    # Calculate term frequencies for query and document
    for token in query_tokens:
        query_vector[token] += 1  # Increment token frequency in the query vector
    for token in doc_tokens:
        doc_vector[token] += 1  # Increment token frequency in the document vector

    # Compute dot product
    dot_product = sum(query_vector[token] * doc_vector[token] for token in query_tokens if token in doc_tokens)

    # Calculate magnitudes
    query_magnitude = math.sqrt(sum(query_vector[token] ** 2 for token in query_tokens))
    doc_magnitude = math.sqrt(sum(doc_vector[token] ** 2 for token in doc_tokens))

    # Avoid division by zero
    if query_magnitude != 0 and doc_magnitude != 0:
        similarity = dot_product / (query_magnitude * doc_magnitude)  # Calculate cosine similarity
        return similarity
    else:
        return 0.0  # Return 0 if either magnitude is zero

# Function to retrieve documents based on similarity
def retrieve_similar_documents(query, inverted_indexes, data):
    query_tokens = tokenize(query)  # Tokenize the query

    # Aggregate similarity scores for each document
    similarity_scores = defaultdict(float)  # Create a dictionary for similarity scores
    for token in query_tokens:
        if token in inverted_indexes:
            postings = inverted_indexes[token]
            for doc_id, relevance in postings:
                doc_tokens = tokenize(data.loc[doc_id, 'courseName'] + " " + 
                                      data.loc[doc_id, 'universityName'] + " " +
                                      data.loc[doc_id, 'city'])
                similarity = cosine_similarity(query_tokens, doc_tokens)
                similarity_scores[doc_id] += similarity * relevance  # Weighted sum of similarities

    # Sort documents by aggregated similarity
    sorted_docs = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_docs

def parse_custom_date(date):
    try:
        return datetime.strptime(date, "%B")  # Add the correct date format for parsing
    except ValueError:
        return pd.NaT  # Return NaT (Not a Timestamp) for invalid dates

def get_current_month_name():
    month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    current_month_number = datetime.now().month
    return month_names[current_month_number - 1]  # Adjust index to match the list

def filter_started_courses(data):
    current_month = datetime.now().month  # Get current month as a number
    data = data.copy()  # Create a copy of the DataFrame to avoid chained assignment warnings
    
    # Omit rows where 'startDate' is NaT and create 'start_months' column
    data = data.dropna(subset=['startDate'])
    data.loc[:, 'start_months'] = data['startDate'].apply(lambda x: [month.strip() for month in x.split(',')])

    def has_started(start_months):
        for month in start_months:
            try:
                month_date = datetime.strptime(month, '%B').month
                if month_date <= current_month:
                    return True
            except ValueError:
                pass  # Ignore invalid month names
        return False

    # Create 'has_started' column
    data.loc[:, 'has_started'] = data['start_months'].apply(has_started)

    # Filter and drop columns if they exist
    if 'start_months' in data.columns and 'has_started' in data.columns:
        data = data[data['has_started']]
        data.drop(columns=['start_months', 'has_started'], inplace=True)
    else:
        print("'start_months' or 'has_started' column not found.")

    return data


def filter_online_modality(data):
    data_copy = data.copy()  # Create a copy of the DataFrame to avoid chained assignment warnings
    return data_copy[data_copy['administration'].str.contains('Online', case=False, na=False, regex=False)]

def search_courses(data, query=None, course_name=None, university_name=None, university_city=None, fee_min=None, fee_max=None, countries=None, started_only=False, online_only=False):
    if query:
    
        # Create a dictionary to hold inverted indexes
        inverted_indexes = {}
        for index, row in data.iterrows():
            # Tokenize relevant fields
            course_tokens = tokenize(row['courseName'])
            university_tokens = tokenize(row['universityName'])
            city_tokens = tokenize(row['city'])
        
            # Update inverted indexes for each token
            for token in course_tokens + university_tokens + city_tokens:
                if token not in inverted_indexes:
                    inverted_indexes[token] = []
                inverted_indexes[token].append((index, 1))  # Assuming relevance of 1 for simplicity
    
        similar_docs = retrieve_similar_documents(query, inverted_indexes, data)
    
        # Retrieve top N documents based on similarity
        top_n = 50  # Number of top documents to retrieve
        top_documents = [doc_id for doc_id, similarity in similar_docs[:top_n]]
        relevant_results = data.loc[top_documents]
        # data = data[data.index.isin(top_documents)]
        data = relevant_results

    # Handling special case for fees (preprocess_fees function needs to be defined)
    data['fees'] = data['fees'].apply(lambda x: 0 if x == "Please see the university website for further information on fees for this course." else x)
    data["fees_USD"] = data["fees"].apply(preprocess_fees)  # Add preprocess_fees function

    # Filtering based on optional queries
    if course_name:
        data = data[data['courseName'].str.contains(course_name, case=False, na=False, regex=False)]
    if university_name:
        data = data[data['universityName'].str.contains(university_name, case=False, na=False, regex=False)]
    if university_city:
        data = data[data['city'].str.contains(university_city, case=False, na=False, regex=False)]

    # Filtering based on fee range
    if fee_min is not None:
        data = data[data['fees_USD'].astype(float).fillna(0) >= fee_min]
    if fee_max is not None:
        data = data[data['fees_USD'].astype(float).fillna(0) <= fee_max]

    # Filtering based on countries array
    if countries:
        data = data[data['country'].isin(countries)]

    # Filter for courses that have started
    if started_only:
        data = filter_started_courses(data)

    # Filter for online modality
    if online_only:
        data = filter_online_modality(data)

    return data

### Example usage
- Pass your optional queries to the search_courses function
- Replace None with the desired search terms

In [55]:
# Example usage: "Computer science courses in Newcastle"
filtered_data = search_courses(data,
                                query="Computer science courses in Newcastle",
                                fee_min=5000,  # Replace with your minimum fee
                                fee_max=15000,  # Replace with your maximum fee
                                countries=["United Kingdom"],  # List of desired countries
                                started_only=True, # Filter for courses that have started in the current month
                                online_only=True)  # Filter for online modality

filtered_data  # Displaying the filtered dataset

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,fees_USD
60,Computer Science MSc (Online),Northumbria University,Distance Learning,MSc,"No matter what your background, drive digital ...","October, January",Full course fees for the academic year 2022/23...,MSc,See Programme Description,Newcastle,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...,11435.0


In [56]:
# Example usage: "Portland universities"
filtered_data = search_courses(data,
                                query="Portland universities",
                                fee_min=None,  # Replace with your minimum fee
                                fee_max=None,  # Replace with your maximum fee
                                countries=[],  # List of desired countries
                                started_only=False, # Filter for courses that have started in the current month
                                online_only=False)  # Filter for online modality

filtered_data  # Displaying the filtered dataset

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,fees_USD
5886,Master of Science in Sport and Performance Psy...,University of Western States,Masters Programs,Full time,The sport and performance psychology (SPP) pro...,"October, April",$587 U.S. per credit hour - 54 credits total -...,MSc,1.5 to 2 years,Portland,USA,Online,https://www.findamasters.com/masters-degrees/c...,587
5835,Master of Science in Human Nutrition and Funct...,University of Western States,Masters Programs,Full time,The University of Western States Master of Sci...,"October, April",$551 U.S. per credit hours - 56 credit hours -...,MSc,8 quarters,Portland,USA,Online,https://www.findamasters.com/masters-degrees/c...,551


In [57]:
# Example usage: courses that have started in the current month
search_results = search_courses(data,
                                course_name=None,
                                university_name=None,
                                university_city=None,
                                fee_min=None,  # Replace with your minimum fee
                                fee_max=None,  # Replace with your maximum fee
                                countries=[],  # List of desired countries
                                started_only=True, # Filter for courses that have started in the current month
                                online_only=False)  # Filter for online modality

# search_results[['courseName', 'universityName', 'url']]
search_results

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,fees_USD
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,0,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,43788.0
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,0,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,0,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,0,MSc,One year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Full time,Register your interest in graduate study at UC...,September,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,17767.0
5996,Materials Chemistry - MSc,University of Bradford,Faculty of Life Sciences,Full time,We provide a unique Master’s education in Mate...,September,0,MSc,1 year full time,Bradford,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
5997,Materials Chemistry MSc,University of Edinburgh,School of Chemistry,Full time,Programme descriptionMaterials Chemistry has e...,September,Tuition fees vary between degree programmes. F...,MSc,1 year full-time,Edinburgh,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,NaN
5998,Materials Engineering,University of Padua,School of Engineering,Full time,The Master's degree Materials Engineering is a...,October,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,On Campus,https://www.findamasters.com/masters-degrees/c...,2956.0


## 6. CommandLine.sh

## Presentation of data:

In [58]:
import pandas as pd

# Read the TSV file into a DataFrame
file_path = 'merged_courses.tsv'
data = pd.read_csv(file_path, sep='\t')

# Display the DataFrame
data

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Full time,Register your interest in graduate study at UC...,September,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5996,Materials Chemistry - MSc,University of Bradford,Faculty of Life Sciences,Full time,We provide a unique Master’s education in Mate...,September,Please see the university website for further ...,MSc,1 year full time,Bradford,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5997,Materials Chemistry MSc,University of Edinburgh,School of Chemistry,Full time,Programme descriptionMaterials Chemistry has e...,September,Tuition fees vary between degree programmes. F...,MSc,1 year full-time,Edinburgh,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5998,Materials Engineering,University of Padua,School of Engineering,Full time,The Master's degree Materials Engineering is a...,October,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,On Campus,https://www.findamasters.com/masters-degrees/c...


## Here is the bash code for CommandLine.sh :

In [ ]:
#!/bin/bash

# Create an empty merged file
touch merged_courses.tsv

for ((i = 1; i <= 6000; i++)); do
  folder="HTML_folders/page_${i}"
  file="html_${i}.html.tsv"

  if [ $i -eq 1 ]; then
    # For the first file, copy the whole content
    cat "${folder}/${file}" >> merged_courses.tsv
  else
    # For files 2 to 6000, omit the first row
    tail -n +2 "${folder}/${file}" >> merged_courses.tsv
  fi
done

printf "The merged_courses.tsv file is generated. \n"

# Question-1:

printf "# Question-1: \n"

# Read 'merged_courses.tsv' file and generate counts for each country
country_count=$(awk -F'\t' 'NR>1 { countries[$11]++ } END { for (country in countries) print "{\"country\": \"" country "\", \"counts\": " countries[country] "}" }' merged_courses.tsv)

# Sort the country count based on counts in descending order
sorted_country_count=$(echo "$country_count" | jq -s 'sort_by(-.counts)')

echo "$sorted_country_count" | jq '.[:5]'

# Extract the value of "country" from the first cell and store it in most_frequent_country
most_frequent_country=$(echo "$sorted_country_count" | jq -r '.[0].country')

echo "Most frequent country: $most_frequent_country"

# Loop over 'merged_courses.tsv' file and filter rows by most frequent country
city_list=$(awk -F'\t' -v most_frequent_country="$most_frequent_country" 'NR>1 && $11 == most_frequent_country {
    cities[$10]++;
}
END {
    for (city in cities) {
        printf "{\"country\": \"%s\", \"city\": \"%s\", \"city_occurrence\": %d}\n", most_frequent_country, city, cities[city];
    }
}' merged_courses.tsv | jq -s '.')

# Sort the city_list based on city_occurrence
sorted_city_list=$(echo "$city_list" | jq 'sort_by(-.city_occurrence)')

echo "$sorted_city_list" | jq '.[:5]'

# Extract the maximum city_occurrence value
max_occurrence=$(echo "$sorted_city_list" | jq '[.[] | .city_occurrence] | max')

# Find cities with the maximum city_occurrence
max_cities=$(echo "$sorted_city_list" | jq --arg max_occurrence "$max_occurrence" "[.[] | select(.city_occurrence == $max_occurrence) | .city]")

printf "\n The most Master's Degrees are in the following cities: "
echo "$max_cities"

# Question-2:

printf "# Question-2: \n"

# Initialize an empty array to store 'part_time' rows
part_time=()

# Read 'merged_courses.tsv' line by line
while IFS=$'\t' read -r col1 col2 col3 isItFullTime col5; do
    # Check if the value in the 'isItFullTime' column is 'Part time'
    if [ "$isItFullTime" = "Part time" ]; then
        # If true, add the entire row to the 'part_time' array
        part_time+=("$col1" "$col2" "$col3" "$isItFullTime" "$col5")
    fi
done < merged_courses.tsv

# Print the length of the 'part_time' array
echo "Number of colleges offering Part-Time education is: ${#part_time[@]}"

# Question-3:

printf "# Question-3: \n"

# Initialize an empty array to store rows containing 'Engineer' in 'courseName'
contain_engineer=()

# Read 'merged_courses.tsv' line by line
while IFS=$'\t' read -r courseName col2 col3 col4 col5; do
    # Check if 'courseName' column contains 'Engineer'
    if [[ "$courseName" == *"Engineer"* ]]; then
        # If true, add the entire row to 'contain_engineer' array
        contain_engineer+=("$courseName" "$col2" "$col3" "$col4" "$col5")
    fi
done < merged_courses.tsv

# Print the length of the 'contain_engineer' array
echo "Length of 'contain_engineer' list: ${#contain_engineer[@]}"

# Calculate the length of the 'contain_engineer' array
length=${#contain_engineer[@]}

# Perform the calculation using bc for floating-point arithmetic
result=$(echo "scale=2; $length / 6000 * 100" | bc)

echo "The percentage of courses in Engineering is: $result%"

### Here is the result:

In [ ]:
(base) armanfeili@Armans-MacBook-Pro ADM-HW3 % bash CommandLine.sh
The merged_courses.tsv file is generated. 
# Question-1: 
[
  {
    "country": "United Kingdom",
    "counts": 4479
  },
  {
    "country": "Ireland",
    "counts": 251
  },
  {
    "country": "Netherlands",
    "counts": 210
  },
  {
    "country": "USA",
    "counts": 182
  },
  {
    "country": "Germany",
    "counts": 125
  }
]
Most frequent country: United Kingdom
[
  {
    "country": "United Kingdom",
    "city": "London",
    "city_occurrence": 1086
  },
  {
    "country": "United Kingdom",
    "city": "Glasgow",
    "city_occurrence": 296
  },
  {
    "country": "United Kingdom",
    "city": "Edinburgh",
    "city_occurrence": 247
  },
  {
    "country": "United Kingdom",
    "city": "Nottingham",
    "city_occurrence": 124
  },
  {
    "country": "United Kingdom",
    "city": "Bristol",
    "city_occurrence": 123
  }
]

 The most Master's Degrees are in the following cities: [
  "London"
]
# Question-2: 
Number of colleges offering Part-Time education is: 3185
# Question-3: 
Length of 'contain_engineer' list: 3075
The percentage of courses in Engineering is: 51.00%

![](./screen-shots/CommandLine-screenshot.png)

## 7. Algorithmic Question
Leonardo is an intern at a company. He is paid based on the total number of hours he has worked. They agreed d days ago that Leonardo could not work less than "mintime" or more than "maxtime" hours per i-th day. Furthermore, he was warned by HR that on his last day at the company, he should provide a detailed report on how many hours he worked each day for the previous d days.

Today is the day Leonardo should report to HR, but the problem is that he didn't account for how many hours he put in for each day, so he only has the total sum of the hours (sumHours) he put in total in these d days. He believes that if he creates a report in which each number "daysHours" corresponds to the total hours he worked on the i-th day while satisfying the HR limitations and the total sum of all "daysHours"  equals "sumHours" he would be fine.

He cannot create such a report independently and requests your assistance. He will give you the number of days "d" , total hours spent "sumHours" , and the HR limitations for each day "i"
, and he wants you to assist him in determining whether it is possible to create such a fake report. If that is possible, make such a report.

Input

The first line of input contains two integers d,  "sumHours"- the number of days Leonardo worked there and the total number of hours he worked for the company. Each of the following d lines contains two integer numbers "mintime" and "maxtime"- the minimum and maximum hours he can work on the "i"'th day.

Output

If such a report cannot be generated, print 'NO' in one output line. If such a report is possible, print 'YES' in the output and d numbers - the number of hours Leonardo spent each day - in the second line. If more than one solution exists, print any of them.

### 1.Implement a code to solve the above mentioned problem.(**following cell is the answer**)

In [61]:
inupt_day_sumhours=input("enter the d and the sumHours: ").split()
d=int(inupt_day_sumhours[0]) #the number of days Leonardo worked
sum_hours=int(inupt_day_sumhours[1]) #the total number of hours he worked for the company

day_limits=list() #creating a list of minimums and maximums of each day he workd
for i in range(d):
    inupt_min_max =input(f"enter the min and max Hours for {i + 1}'th' day : ").split()
    inupt_min=int(inupt_min_max[0])
    inupt_max=int(inupt_min_max[1])
    day_limits.append(([] if inupt_min == 0 else [0]) + [i for i in range(inupt_min, inupt_max + 1)])
    
results=list()
from itertools import product
all_combinations = list(product(*day_limits))
for combination in all_combinations:
    if sum(combination)==sum_hours:
        results.append(combination)
if len(results)==0:
    print("NO")
else:
    print("YES")
    for i in results:
        print(*i)

enter the d and the sumHours:  4 18
enter the min and max Hours for 1'th' day :  0 2
enter the min and max Hours for 2'th' day :  3 6
enter the min and max Hours for 3'th' day :  19 22
enter the min and max Hours for 4'th' day :  13 18


YES
0 0 0 18
0 3 0 15
0 4 0 14
0 5 0 13
1 0 0 17
1 3 0 14
1 4 0 13
2 0 0 16
2 3 0 13


##### 2.What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.
well the time complexity of my code is elaborated as below,line by line:
##### A.
the firt parsing input(the first 3 lines):Tis has a constant time complexity of O(1).because it performs a fixed number of operations regardless of the input size.
##### B.
Creating day_limits List(the next 6lines):The loop that fills the day_limits list iterates d times, and as we know the d is the number of days Leonardo worked. In each iteration, it performs a constant number of operations. Therefore, the time complexity of this part is O(d) and because days are a few it will be the same as o("a small number like=4").
##### C.
Generating All Combinations of the given intervals: we are using the product function from the itertools module to generate all combinations of hours for each day. The number of combinations is tied to the number of options for each day. In the worst case scenario, this results in a total of (max-min+1)^d combinations. The time complexity of generating all combinations is O((max-min+1)^d), where max and min are the maximum and minimum hours for a day, respectively.
##### D.
Checking Combinations and Printing Results: The loop that iterates over all combinations to check if the sum of each combination tuple is equal to sum_hours has a time complexity of O((max-min+1)^d) since it loops through all combinations.
Printing the results has a time complexity proportional to the number of results, which can be at most O((max-min+1)^d) in the worst case scenario.

![](./screen-shots/Screen-Shot-time-complexity.png)

#### 4.What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

regarding the optimality of my code, it can be optimal just for a few entered days(d).Due to the fact that it has an exponential time complexity, it is not optimal for big numbers of "d". Let me elaborate on my response: at the very first attempts, i decided to do this assignment with many different loops to get the answer. but my code had problems in its algorithm which just checked two successive days and it was not correct.then I googled my question to see how I could go through a list of some tuples to find the SumHour out of each element of these nested tuples in an optimal way. I figured out that this problem should be resolved by implementing a Greedy Algorithm... I searched about it and got into its functionality. Afterward, using the Greedy algorithm, I came up with an algorithm that worked But just printed out one soulution and also i was not worked for all inputs and as it was mentioned in the question,the code has to print all the possible solutions to this problem and not just one solution.ultimately,i came up with this algorithm which can be turned into a more optimal by using greedy and recursive form of algorithms at the same time to generate the right report and print all the possible outcomes but it is not working for some inputs.. i just mention it but i still believe on my algorithm for thorough and correct answer than this one which i wrote by the help of CHatGPT.

In [62]:
# grabing the Inputs:
inupt_day_sumhours=input("enter the d and eh sumHours: ").split()
d=int(inupt_day_sumhours[0]) #the number of days Leonardo worked
sum_hours=int(inupt_day_sumhours[1]) #the total number of hours he worked for the company

day_limits=list() #creating a list of minimums and maximums of each day he workd
for i in range(d):
    inupt_min_max = input(f"enter the min and max Hours for {i + 1}'th' day : ").split()
    inupt_min=int(inupt_min_max[0])
    inupt_max=int(inupt_min_max[1])
    day_limits.append([inupt_min,inupt_max])
#if i want to print all the possible outcomes, then i have to create a function
# to call it again and again which is called recursive function
def generate_all_reports(d, sum_hours, day_limits, temporary_report=None, index=0):
    #first it is needed to chech whether this is the first report or not
    #if it is the first one, we should create a report list
    if temporary_report is None:
        temporary_report = [0] * d
    #we need to chech wether we get to the last day to terminate our recursive report generationg process
    if index == d:
        if sum(temporary_report) == sum_hours: #here all the temporary lists which have our
            return [temporary_report.copy()]   #condition is being seperated for the final result
        else:
            return []

    #this block of code is the recursive generation of all valid reports. It explores different
    #combinations of hours for the current day and recursively explores all possible outcomes for the remaining
    #days. The results are piled up in the reports list, which is then returned.
    reports = list()
    for hours in range(day_limits[index][0], min(day_limits[index][1] + 1, sum_hours - sum(temporary_report) + 1)):
        temporary_report[index] = hours
        remaining_reports = generate_all_reports(d, sum_hours, day_limits, temporary_report.copy(), index + 1)
        reports.extend(remaining_reports)
    return reports

#here we call our functioon
all_reports = generate_all_reports(d, sum_hours, day_limits)
#and last nut not least, we print our final result
if not all_reports:
    print("NO")
else:
    print("YES")
    for report in all_reports:
        print(*report)

enter the d and eh sumHours:  3 17
enter the min and max Hours for 1'th' day :  0 2
enter the min and max Hours for 2'th' day :  3 6
enter the min and max Hours for 3'th' day :  5 15


YES
0 3 14
0 4 13
0 5 12
0 6 11
1 3 13
1 4 12
1 5 11
1 6 10
2 3 12
2 4 11
2 5 10
2 6 9


In [64]:
day_limits=list() #creating a list of minimums and maximums of each day he workd
for i in range(d):
    inupt_min_max = input(f"enter the min and max Hours for {i + 1}'th' day : ").split()
    inupt_min=int(inupt_min_max[0])
    inupt_max=int(inupt_min_max[1])
    day_limits.append([inupt_min,inupt_max])

#here we call our functioon
all_reports = generate_all_reports(d, sum_hours, day_limits)
#and last nut not least, we print our final result
if not all_reports:
    print("NO")
else:
    print("YES")
    for report in all_reports:
        print(*report)

enter the min and max Hours for 1'th' day :  1 6
enter the min and max Hours for 2'th' day :  1 1
enter the min and max Hours for 3'th' day :  5 6


NO
